In [ ]:
# Saving the DataFrame as an Excel file
df_parquet.to_excel("falcon_op.xlsx", index=False)

In [ ]:
import pandas as pd

CODE FOR EVALAUTION:

In [ ]:
# Load the ground truth and model output Excel files
file_path_ground_truth = 'final_NER_answers.xlsx'
file_path_model_output = 'final_falcon_answers.xlsx'

ground_truth_data = pd.read_excel(file_path_ground_truth)
model_output_data = pd.read_excel(file_path_model_output)



In [ ]:
# Convert 'Entity_name' columns in both datasets to lowercase for case-insensitive comparison
ground_truth_data['Entity_name_lower'] = ground_truth_data['Entity_name'].str.lower()
model_output_data['Entity_name_lower'] = model_output_data['Entity_name'].str.lower()

# Merging the data on 'Abstract Number' and the lowercase 'Entity_name'
comparison_data = pd.merge(ground_truth_data, model_output_data,
                           left_on=['Abstract Number', 'Entity_name_lower'],
                           right_on=['Abstract Number', 'Entity_name_lower'],
                           how='outer')

# Dropping the lowercase 'Entity_name' column used for merging
comparison_data.drop('Entity_name_lower', axis=1, inplace=True)

# Filtering out rows where ground truth is missing (keeping rows where model output is missing)
comparison_data_filtered = comparison_data.dropna(subset=['Entity_class_x'])

# Replacing missing values in the model output column with 'NoPrediction'
comparison_data_filtered['Entity_class_y'].fillna('NoPrediction', inplace=True)

# Calculating precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(comparison_data_filtered['Entity_class_x'],
                                                                 comparison_data_filtered['Entity_class_y'],
                                                                 average='weighted',
                                                                 zero_division=0)

precision, recall, f1_score

<ipython-input-61-66411febbd8d>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comparison_data_filtered['Entity_class_y'].fillna('NoPrediction', inplace=True)


(0.21851851851851853, 0.004285714285714286, 0.007509253095459993)

In [ ]:
# Grouping the filtered data by Abstract Number
grouped_data = comparison_data_filtered.groupby('Abstract Number')

# Initializing lists to store precision, recall, and F1 score for each abstract
precision_list = []
recall_list = []
f1_score_list = []

# Calculating precision, recall, and F1 score for each abstract
for _, group in grouped_data:
    precision, recall, f1_score, _ = precision_recall_fscore_support(group['Entity_class_x'],
                                                                     group['Entity_class_y'],
                                                                     average='weighted',
                                                                     zero_division=0)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_score_list.append(f1_score)

# Averaging the precision, recall, and F1 score across all abstracts
average_precision = sum(precision_list) / len(precision_list)
average_recall = sum(recall_list) / len(recall_list)
average_f1_score = sum(f1_score_list) / len(f1_score_list)

average_precision, average_recall, average_f1_score

(0.012714285714285713, 0.008428571428571428, 0.009285714285714286)